In [1]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

from pydantic import BaseModel
import markdown_to_json
import json


In [2]:
llm = Ollama(
    model="agent_pm", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

llm_json = Ollama(
    model="agent_json_fixer", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

In [30]:
class ResponseComponent(BaseModel):
    name: str
    description: str
    output_format: str

tasks = ResponseComponent(
        name="tasks",
        description="the project split into tasks",
        output_format="markdown list",
    )
timeline = ResponseComponent(
        name="timeline",
        description="the time required for each task",
        output_format="markdown list",
    )
deliverables =  ResponseComponent(
        name="deliverables",
        description="the project deliverables with scope",
        output_format="markdown list",
    )
team = ResponseComponent(
        name="team",
        description="the required project team's skills and experience level",
        output_format="markdown list",
    )
risks = ResponseComponent(
        name="risks",
        description="the project risks",
        output_format="markdown list",
    )
budget = ResponseComponent(
        name="budget",
        description="the budget for each task",
        output_format="markdown list",
    )
metrics = ResponseComponent(
        name="metrics",
        description="the project metrics",
        output_format="markdown list",
    )

def structured_output_instructions_from_response_components(response_components: list[ResponseComponent]):
    base_instruction = """\
The output should be a markdown code snippet formatted in the following schema:
"""
    for n, response_component in enumerate(response_components):
        base_instruction += f"{n+1}. A markdown heading (#) with the title {response_component.name}, followed by a desciption of {response_component.description} as a {response_component.output_format}.\n"
    base_instruction += "Do not return a response as a multilevel list when lists are used.\n"
    
    return base_instruction

response_components = [
    tasks,
    timeline,
    deliverables,
    team,
    risks,
    budget,
    metrics,
]

response_format = structured_output_instructions_from_response_components(response_components)

print(response_format)

The output should be a markdown code snippet formatted in the following schema:
1. A markdown heading (#) with the title tasks, followed by a desciption of the project split into tasks as a markdown list.
2. A markdown heading (#) with the title timeline, followed by a desciption of the time required for each task as a markdown list.
3. A markdown heading (#) with the title deliverables, followed by a desciption of the project deliverables with scope as a markdown list.
4. A markdown heading (#) with the title team, followed by a desciption of the required project team's skills and experience level as a markdown list.
5. A markdown heading (#) with the title risks, followed by a desciption of the project risks as a markdown list.
6. A markdown heading (#) with the title budget, followed by a desciption of the budget for each task as a markdown list.
7. A markdown heading (#) with the title metrics, followed by a desciption of the project metrics as a markdown list.
Do not return a resp

In [31]:
template = """\
You have been given the following project brief. Identify and plan the key project tasks step by step.

{format_instructions}

Project Brief:
{brief}
"""

brief = """\
- The project is for a big multinational firm.
- The firm employs about 10000 people.
- The firm is grouped into a hierarchical structure, where each group specialises in a different industry or competency.
- The firm is struggling to forecast its revenue accurately.
- We are proposing to use machine learning an algorithms coupled with sourcing external market data to improve forecasting.
- We aim to achieve a better accuracy than their current forecasting process.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["brief"],
    partial_variables={"format_instructions": response_format}
)

_input = prompt.format_prompt(brief=brief)
output_plan = llm(_input.to_string())

## Tasks:

1. Define the problem and identify the key factors affecting revenue forecasting.
	* Research and analyze the current forecasting process of the firm.
	* Identify the sources of external market data that can be used to improve forecasting accuracy.
2. Collect and preprocess the external market data.
	* Determine the relevant market data sources for the firm's industries or competencies.
	* Clean and normalize the data to ensure consistency and accuracy.
3. Develop a machine learning model to predict revenue.
	* Choose appropriate algorithms and techniques based on the problem requirements.
	* Train the model using the preprocessed market data and the firm's historical revenue data.
4. Evaluate and refine the model.
	* Test the model with a holdout sample to evaluate its performance.
	* Identify areas for improvement and refine the model as needed.
5. Implement the model in the firm's forecasting process.
	* Integrate the model into the existing forecasting system or develop 

In [64]:
output_plan_dict = markdown_to_json.dictify(output_plan)


def find_headings(response_components: ResponseComponent, text: str):
    if type(text) != str:
        return None
    text_to_test = text.lower()
    for component in response_components:
        if component.name.lower() in text_to_test and len(text_to_test) <= len(component.name)*2:
            return component.name

result_dict = {}
last_heading = None
for l in output_plan_dict:
    result = find_headings(response_components, l)
    if result is not None:
        result_dict[result] = output_plan_dict[l]

def flatten_list(the_list: list, nr_prefix=''):
    new_list = []
    n = 0
    for list_item in the_list:  
        if type(list_item) == list:
            number_str = f"{nr_prefix}{str(n)}."
            new_list.extend(flatten_list(list_item, nr_prefix=number_str))
        else:
            n += 1
            number_str = f"{nr_prefix}{str(n)}."
            new_list.append(f"{number_str} {str(list_item)}")
    return new_list

for k, v in result_dict.items():
    result_dict[k] = flatten_list(v)
    # print(k, len(result_dict[k]))


tasks 18
timeline 6
deliverables 4
team 6
risks 5
budget 6
metrics 4


In [67]:
result_dict['tasks']

['1. Project manager with experience in managing complex projects and coordinating teams.',
 '2. Data scientist with expertise in machine learning and data analysis.',
 '3. Data analyst with experience in collecting, preprocessing, and evaluating market data.',
 "4. Business analyst with knowledge of the firm's industries or competencies and their revenue forecasting processes.",
 '5. Software developer with experience in integrating machine learning models into existing systems.',
 '6. Quality assurance specialist to ensure that the model is accurate and reliable.']

In [ ]:
# print(json.dumps(output_plan_dict, indent=2))

In [68]:
template = """\
Given the following project brief, and a list of tasks to solve for the brief. Take each task in the list and plan it step by step in detail.

{format_instructions}

Project Brief:
{brief}

Tasks:
{tasks}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["brief"],
    partial_variables={"format_instructions": response_format}
)

_input = prompt.format_prompt(brief=brief, tasks=result_dict['tasks'])
task_plan = llm(_input.to_string())

# Tasks

## Define the problem and identify key factors affecting revenue forecasting

* Identify the current forecasting process of the firm and its accuracy
* Determine the sources of external market data that can be used to improve forecasting accuracy
* Identify the relevant industries or competencies of the firm for which market data should be collected

## Research and analyze the current forecasting process of the firm

* Evaluate the current forecasting methodology and tools used by the firm
* Determine the strengths, weaknesses, opportunities, and threats (SWOT analysis) of the firm's forecasting process
* Identify areas for improvement in the current forecasting process

## Identify sources of external market data that can be used to improve forecasting accuracy

* Determine the relevant market data sources for the firm's industries or competencies
* Evaluate the quality and relevance of each market data source
* Identify any potential biases or limitations in the market data

KeyboardInterrupt: 